In [14]:
import sys
import pandas as pd
from datetime import date
import warnings

sys.path.insert(0, '..\DATA')

import INIT_STORE as I_S

In [25]:
#load data 
warnings.simplefilter(action = 'ignore', category = UserWarning)

cnxn = I_S.connect_cloud_sql('PAIRS', I_S.GCLOUD_PWD, I_S.GCLOUD_IP,
    I_S.SSL_CA_PATH, I_S.SSL_CERT_PATH, I_S.SSL_KEY_PATH)
pairs = I_S.get_stored_pairs()

is_first = True
for pair in pairs:
    sql = """
    SELECT 
        time, volume, bid_c
    FROM {}
    """.format(pair)
    df_mid = pd.read_sql(sql, cnxn) 
    df_mid.columns = ['{}{}'.format(c, '_{}'.format(pair))
             for c in df_mid.columns]

    df_mid.set_index(pd.to_datetime(df_mid[df_mid.columns[0]]), inplace=True)

    if not is_first:
        df = merge=pd.merge(df,df_mid, how='inner', left_index=True, right_index=True)
    else:
        df = df_mid
        is_first = False

cnxn.close()
print(df.head)
df.to_csv("LOCAL_COPY_PAIRS_".format(str(date.today())))

<bound method NDFrame.head of                            time_GBP_AUD  volume_GBP_AUD  bid_c_GBP_AUD  \
2021-09-22 11:00:00 2021-09-22 11:00:00            4953        1.88129   
2021-09-22 12:00:00 2021-09-22 12:00:00            7097        1.88037   
2021-09-22 13:00:00 2021-09-22 13:00:00           10941        1.88127   
2021-09-22 14:00:00 2021-09-22 14:00:00           11070        1.87986   
2021-09-22 15:00:00 2021-09-22 15:00:00            9265        1.88190   
...                                 ...             ...            ...   
2022-07-12 18:00:00 2022-07-12 18:00:00            4339        1.75852   
2022-07-12 19:00:00 2022-07-12 19:00:00            6327        1.75843   
2022-07-12 20:00:00 2022-07-12 20:00:00            2390        1.75889   
2022-07-12 21:00:00 2022-07-12 21:00:00             355        1.75859   
2022-07-12 22:00:00 2022-07-12 22:00:00            2052        1.75904   

                           time_EUR_USD  volume_EUR_USD  bid_c_EUR_USD  
2021-09-